In [17]:
# Add Matplotlib inline magic command
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import os
import matplotlib as mpl 
from functools import reduce

In [2]:
# Files to load
city_data_to_load = os.path.join("Resources","city_data.csv")
ride_data_to_load = os.path.join("Resources","ride_data.csv")

In [3]:
# Read the city data file and store it in a pandas DataFrame.
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(10)

,city,driver_count,type
0,Richardfort,38,Urban
1,Williamsstad,59,Urban
2,Port Angela,67,Urban
3,Rodneyfort,34,Urban
4,West Robert,39,Urban
5,West Anthony,70,Urban
6,West Angela,48,Urban
7,Martinezhaven,25,Urban
8,Karenberg,22,Urban
9,Barajasview,26,Urban


In [4]:
# Read the ride data file and store it in a pandas DataFrame.
ride_data_df = pd.read_csv(ride_data_to_load)
ride_data_df.head(10)

,city,date,fare,ride_id
0,Lake Jonathanshire,2019-01-14 10:14:22,13.83,5739410935873
1,South Michelleport,2019-03-04 18:24:09,30.24,2343912425577
2,Port Samanthamouth,2019-02-24 04:29:00,33.44,2005065760003
3,Rodneyfort,2019-02-10 23:22:03,23.44,5149245426178
4,South Jack,2019-03-06 04:28:35,34.58,3908451377344
5,South Latoya,2019-03-11 12:26:48,9.52,1994999424437
6,New Paulville,2019-02-27 11:17:56,43.25,793208410091
7,Simpsonburgh,2019-04-26 00:43:24,35.98,111953927754
8,South Karenland,2019-01-08 03:28:48,35.09,7995623208694
9,North Jasmine,2019-03-09 06:26:29,42.81,5327642267789


In [112]:
# Combine the data into a single dataset
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Display the DataFrame
pyber_data_df.head()

,city,date,fare,ride_id,driver_count,type
0,Lake Jonathanshire,2019-01-14 10:14:22,13.83,5739410935873,5,Urban
1,South Michelleport,2019-03-04 18:24:09,30.24,2343912425577,72,Urban
2,Port Samanthamouth,2019-02-24 04:29:00,33.44,2005065760003,57,Urban
3,Rodneyfort,2019-02-10 23:22:03,23.44,5149245426178,34,Urban
4,South Jack,2019-03-06 04:28:35,34.58,3908451377344,46,Urban


In [114]:
#create data frames for the summaries
total_rides = pyber_data_df.groupby(pyber_data_df["type"]).count()["ride_id"].rename("Total Rides")
total_drivers = city_data_df.groupby(city_data_df["type"]).sum()["driver_count"].rename("Total Drivers")
total_fares = pyber_data_df.groupby(pyber_data_df["type"]).sum()["fare"].rename("Total Fares")
avg_fare_per_ride = pyber_data_df.groupby(pyber_data_df["type"]).mean()["fare"].rename("Average Fare Per Ride")
avg_fare_per_driver = (total_fares/total_drivers).rename("Average Fare Per Driver")

total_drivers

type
Rural         78
Suburban     490
Urban       2405
Name: Total Drivers, dtype: int64

In [123]:
dfs_to_concat = [total_rides,total_drivers,total_fares,avg_fare_per_ride, avg_fare_per_driver]
final_summary_df = pd.concat(dfs_to_concat, axis=1)
#final_summary_df["Average Fare Per Driver"] = final_summary_df["Total Fares","Total Drivers"].divide(axis=1)
#final_summary_df["Average Fare Per Driver"] = final_summary_df.map(final_summary_df["Total Fares"]/final_summary_df"Total Drivers"])


final_summary_df.style.format({"Total Rides": "{:,}", 
                               "Total Drivers": "{:,}", 
                               "Total Fares": "${:20,.2f}",
                               "Average Fare Per Ride":"${:20,.2f}",
                               "Average Fare Per Driver":"${:20,.2f}"})


,Total Rides,Total Drivers,Total Fares,Average Fare Per Ride,Average Fare Per Driver
type,,,,,
Rural,125,78,"$ 4,327.93",$ 34.62,$ 55.49
Suburban,625,490,"$ 19,356.33",$ 30.97,$ 39.50
Urban,"1,625","2,405","$ 39,854.38",$ 24.53,$ 16.57
